- Pas de couleur pour que ça passe sur github (sorry!)
- Qu'est ce qui passe pas bien à l'échelle ? Le c mixture component est un peu lourd, en $\mathcal{O}(nk)$
- comment ça se passe si X est multi (voire grande) dimensionnel?
- pourquoi ça overshoot ?
- est-ce qu'on utilise toujours ce type de parametrisation ou ça peut changer ?


# <font color=darkcyan> Variational inference </font>
#### <font color=darkorange>Basics: Evidence Lower Bound (ELBO) & Coordinate Ascent Variational Inference (CAVI) </font>

In [ ]:
"""""""""""""""""
Required packages
"""""""""""""""""
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

<font color=darkred>Mixture of Gaussian distributions.</font>

Following ``Variational Inference: A Review for Statisticians, Blei et al; (2017)``, consider a Bayesian mixture of unit-variance univariate Gaussians. 

There are $K$ mixture components, each component $k$ of the mixture is a Gaussian distribution with mean $\mu_k$ and variance 1. The random variables $\mu = (\mu_k)_{1\leqslant k \leqslant K}$ are assumed to be independent and identically distributed (i.i.d.) with Gaussian distribution with mean 0 and variance $\sigma^2$. The weight of each mixture component $k$ written $\omega_k$. Conditionally on $\mu$, the observations $(X_i)_{1\leqslant i\leqslant n}$ are assumed to be i.i.d. with probability density:

$$
p(x|\mu) = \sum_{k=1}^K \omega_k \varphi_{\mu_k,1}(x)\,,
$$

where $\varphi_{\mu_k,\sigma^2}$ is the Gaussian probability function with mean $\mu_k$ and variance $\sigma^2$. The likelihood is then given by:

$$
p(x_1,\cdots,x_n) = \int p(x_1,\cdots,x_n|\mu) p(\mu) \mathrm{d} \mu = \int \prod_{i=1}^n p(x_i|\mu) p(\mu) \mathrm{d} \mu = \int \prod_{i=1}^n \left(\sum_{k=1}^K \omega_k \varphi_{\mu_k,1}(x_i)\right) p(\mu) \mathrm{d} \mu
$$

Application with $K= 3$, $\sigma^2 = 5$, $\omega_k = 1/K$ for all $1\leqslant k \leqslant K$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def eightgaussian(n_points):
    """
     Returns the eight gaussian dataset.
    """
    n = np.random.randint(0,8, n_points)
    noisex = np.random.normal(size=(n_points)) * 0.2
    noisey = np.random.normal(size=(n_points)) * 0.2
    x_centers,y_centers = [np.cos(n* np.pi/4.0) * 5 + noisex, np.sin(n* np.pi/4.0) * 5 + noisey]
    return np.vstack((x_centers,y_centers)).T
            
X = eightgaussian(1000)
X.shape
plt.figure(figsize=(6,6))
plt.scatter(X[:,0], X[:,1], s=1);
plt.axis('square');

Our aim is to approximate the posterior distribution $p(\mu,c|x)$ where $c = (c_1,\cdots,c_n)$ are the mixture components of the observations.  The mean-field variational family is described as follows:

$$
q(\mu,c) = \prod_{k=1}^K \varphi_{m_k,s_k}(\mu_k)\prod_{i=1}^n \mathrm{Cat}_{\phi_i}(c_i)\,, 
$$

which means that:

- $\mu$ and $c$ are independent.
- $(\mu_{k})_{1\leqslant k \leqslant K}$ are independent with Gaussian distribution with means $(m_{k})_{1\leqslant k \leqslant K}$ and variances $(s_{k})_{1\leqslant k \leqslant K}$.
- $(c_{i})_{1\leqslant i \leqslant n}$ are independent with multinomial distribution with parameters $(\phi_i)_{1\leqslant i \leqslant n}$: $q(c_i=k) = \phi_i(k)$ for $1\leqslant k \leqslant K$. 

Write $\mathcal{D}$ the family of such distributions when $(m_{k})_{1\leqslant k \leqslant K}\in \mathbb{R}^K$, variances $(s_{k})_{1\leqslant k \leqslant K}\in (\mathbb{R}_+^*)^K$ and $(\phi_i)_{1\leqslant i \leqslant n}\in \mathcal{S}_K^n$ where $\mathcal{S}_K$ is the $K$-dimensional probability simplex. 

The objective is now to find the
"best candidate" in $\mathcal{D}$ to approximate $p(\mu,c|x)$, i.e. the one ``which minimizes the following KL divergence``:

$$
q^* = \mathrm{Argmin}_{q\in\mathcal{D}} \mathrm{KL}\left(q(\mu,c)\|p(\mu,c|x)\right)\,.
$$

Note that
\begin{align*}
\mathrm{KL}\left(q(\mu,c)\|p(\mu,c|x)\right) &= \mathbb{E}_q[\log q(\mu,c)] - \mathbb{E}_q[\log p(\mu,c|x)]\,,\\
 &= \mathbb{E}_q[\log q(\mu,c)] - \mathbb{E}_q[\log p(\mu,c,x)]+\log p(x)\,,\\
&= -\mathrm{ELBO}(q)+\log p(x)\,,
\end{align*}

where the ``Evidence Lower Bound`` (ELBO) is

$$
\mathrm{ELBO}(q) = -\mathbb{E}_q[\log q(\mu,c)] + \mathbb{E}_q[\log (\mu,c,x)] \,.
$$

Therefore, ``minimizing the KL divergence`` boils down to maximizing the ELBO, where $\log p(x)\geqslant \mathrm{ELBO}(q)$.

The complexity of the family $\mathcal{D}$ determines the complexity of the optimization.

<font color=darkred>Generic CAVI algorithm.</font>

Detail here why the algorithm computes iteratively for all $1\leqslant k \leqslant K$,

$$
q(\mu_k) \propto \mathrm{exp}\left(\mathbb{E}_{\tilde q_{\mu_k}}[\log \tilde p_k(\mu_k|x)]\right)
$$

and for all $1\leqslant i \leqslant n$,

$$
q(c_i) \propto \mathrm{exp}\left(\mathbb{E}_{\tilde q_{c_i}}[\log \tilde p_i(c_i|x)]\right)\,,
$$

where 

- $\tilde p_i(c_i|x)$ is the conditional distribution of $c_i$ given the observations and all the other parameters and $\tilde p_k(\mu_k|x)$ is the conditional distribution of $\mu_k$ given the observations and all the other parameters.

- $\mathbb{E}_{\tilde q_z}$ is the expectation under the variational law of all parameters except $z$.

<font color=darkred>Application to the mixture of Gaussian distributions.</font>

As $\tilde p_i(c_i|x)$ be the conditional distribution of $c_i$ given the observations and the other parameters.

$$
\tilde p_i(c_i|x) \propto p(c_i)p(x_i|c_i,\mu) \propto p(c_i)\prod_{k=1}^K \left(\varphi_{\mu_k,1}(x_i)\right)^{1_{c_i=k}}\,. 
$$

Therefore,

$$
\mathbb{E}_{\tilde q_{c_i}}[\log \tilde p_i(c_i|x)] = \log p(c_i) + \sum_{k=1}^K 1_{c_i=k} \mathbb{E}_{\tilde q_{c_i}}[\log \varphi_{\mu_k,1}(x_i)]
$$

and

\begin{align*}
\mathrm{exp}\left(\mathbb{E}_{\tilde q_{c_i}}[\log \tilde p_i(c_i|x)]\right) &\propto p(c_i) \mathrm{exp}\left(\sum_{k=1}^K 1_{c_i=k} \mathbb{E}_{\tilde q_{c_i}}[\log \varphi_{\mu_k,1}(x_i)]\right)\,\\
&\propto p(c_i) \mathrm{exp}\left(\sum_{k=1}^K 1_{c_i=k} \mathbb{E}_{\tilde q_{c_i}}[-(x_i-\mu_k)^2/2]\right)\,\\
&\propto p(c_i) \mathrm{exp}\left(\sum_{k=1}^K 1_{c_i=k} \mathbb{E}_{\tilde q_{c_i}}[-(x_i-\mu_k)^2/2]\right)\,.
\end{align*}

The update is then written:

$$
\varphi_i(k) \propto p(c_i=k) \mathrm{exp}\left(m_k x_i - \frac{m^2_k + s_k}{2}\right)\,.
$$

<font color=darkred> Update of $(\phi_i)_{1\leqslant i \leqslant n}$ using CAVI.</font>

In [ ]:
def CAVI_update_phi(X,m,s2):
    
    first_term_mean = np.dot(X, m.T)
    second_term_mean = -(m**2 + s2).sum(axis=-1)/2
    
    phi = np.exp(first_term_mean + second_term_mean)
    phi = phi / phi.sum(1)[:, np.newaxis]
    
    return phi

As $\tilde p_k(\mu_k|x)$ be the conditional distribution of $\mu_k$ given the observations and the other parameters.

$$
\tilde p_k(\mu_k|x) \propto p(\mu_k)\prod_{i=1}^np(x_i|c_i,\mu) \propto p(\mu_k)\prod_{i=1}^n p(x_i|\mu,c_i)\,. 
$$

Therefore,

$$
\mathbb{E}_{\tilde q_{\mu_k}}[\log \tilde p_k(\mu_k|x)] = \log p(\mu_k) + \sum_{i=1}^n \mathbb{E}_{\tilde q_{\mu_k}}[\log p(x_i|\mu,c_i)]
$$

and

\begin{align*}
\mathrm{exp}\left(\mathbb{E}_{\tilde q_{\mu_k}}[\log \tilde p_i(c_i|x)]\right) &\propto p(\mu_k) \mathrm{exp}\left(\sum_{i=1}^n\sum_{k=1}^K  \mathbb{E}_{\tilde q_{\mu_k}}[1_{c_i=k}\log \varphi_{\mu_k,1}(x_i)]\right)\,\\
&\propto p(\mu_k) \mathrm{exp}\left(\sum_{i=1}^n \phi_i(k) \mathbb{E}_{\tilde q_{\mu_k}}[\log \varphi_{\mu_k,1}(x_i)]\right)\,\\
&\propto \mathrm{exp}\left(-\frac{\mu_k^2}{2\sigma^2}-\frac{1}{2}\sum_{i=1}^n \phi_i(k)(x_i-\mu_k)^2\right)\,,\\
&\propto \mathrm{exp}\left(-\frac{\mu_k^2}{2\sigma^2}+\sum_{i=1}^n \phi_i(k)x_i\mu_k - \frac{1}{2}\sum_{i=1}^n \phi_i(k)\mu^2_k\right)\,.
\end{align*}

The update is then written:

$$
\mu_k = \frac{\sum_{i=1}^n \phi_i(k)x_i}{1/\sigma^2 + \sum_{i=1}^n \phi_i(k)}\quad\mathrm{and}\quad s_k = \frac{1}{1/\sigma^2 + \sum_{i=1}^n \phi_i(k)}\,. 
$$

<font color=darkred> Update of $(m_{k})_{1\leqslant k \leqslant K}$ and $(s_{k})_{1\leqslant k \leqslant K}$ using CAVI.</font>

In [ ]:
def CAVI_update_mu_s2(X,m,phi,s2,sigma2,K):
    
    phi_sum = phi.sum(0)[:,np.newaxis]
    m  = np.dot(phi.T,X) * (1/sigma2 + phi_sum)**(-1)
    s2 = (1/sigma2 + phi_sum)**(-1)
    print(f"m {m.shape}, s2 {s2.shape}")
    return m, s2

In [ ]:
def elbo(X,phi,m,s2,sigma2):
        
        first_term  = (np.log(s2) - m/sigma2).sum()
        second_term = (-0.5*np.dot(X**2, (s2+m**2).T) + np.dot(X, m.T) - np.log(phi))*phi

        return first_term + second_term.sum()

In [ ]:
def CAVI_mixture_Gaussian(X,m, s2, phi, sigma2, max_iter = 500, epsilon = 1e-8):
        
        elbos  = [elbo(X,phi,m,s2,sigma2)]
        m_est  = [m]
        s2_est = [s2]
        
        for it in range(1, max_iter+1):
            
            phi   = CAVI_update_phi(X,m,s2)
            m, s2 = CAVI_update_mu_s2(X,m,phi,s2,sigma2,K)
            
            m_est.append(m)
            s2_est.append(s2)
            
            elbos.append(elbo(X,phi,m,s2,sigma2))

            if np.abs(elbos[-2] - elbos[-1]) <= epsilon:
                break
        
        return elbos, m_est, s2_est

<font color=darkred> Run the algorithm.</font>

In [ ]:
# Initialization
K = 9

phi_init = np.random.dirichlet([np.random.random()*np.random.randint(1, 10)]*K, n_samples)
m_init = np.random.normal(0, 1, (K,2))
s2_init = np.random.random((K,2))

In [ ]:
fig = plt.figure(figsize=(8,5))
elbos, m_est, s2_est = CAVI_mixture_Gaussian(X, m_init, s2_init, phi_init, np.array([3., 3.]), 500, 0.001)
plt.plot(np.array(elbos),label = 'ELBO')
plt.tick_params(labelright=True)
plt.grid(True)
plt.xlabel('Iterations')
plt.legend();

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(X[:,0], X[:,1], s=1);
plt.scatter(m_est[-1][:,0],m_est[-1][:,1], s=50, c="r")
plt.axis('square');

In [ ]:
m_est[-1], s2_est[-1]